# Classification with Decision Trees and Naive Bayes 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Import and pre-process the data using `pandas`.
Data source: https://archive-beta.ics.uci.edu/dataset/73/mushroom

For practicality, it also on github (csv format, with features names): https://github.com/GaiaSaveri/intro-to-ml/blob/main/data/Mushroom.csv

In [3]:
FFILE = './Mushrooms.csv'
if os.path.isfile(FFILE): 
    print("File already exists")
    if os.access(FFILE, os.R_OK):
        print ("File is readable")
    else:
        print ("File is not readable, removing it and downloading again")
        !rm FFILE
        !wget "https://raw.githubusercontent.com/GaiaSaveri/intro-to-ml/main/data/Mushroom.csv"
else:
    print("Either the file is missing or not readable, download it")
    !wget "https://raw.githubusercontent.com/GaiaSaveri/intro-to-ml/main/data/Mushroom.csv"

Either the file is missing or not readable, download it


"wget" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


Divide features and labels and split the dataset into train and test.

In [4]:
train_data_m=pd.read_csv("./Mushroom.csv")
train_data_m.head()  # viewing some row of the dataset

,CLASS,CAP-SHAPE,CAP-SURF,CAP-COLOR,BRUISES,ODOR,GILL-ATTACH,GILL-SPACE,GILL-SIZE,GILL-COLOR,...,STALK-SURF-BELOW,STALK-COLOR-ABOVE,STALK-COLOR-BELOW,VEIL-TYPE,VEIL-COLOR,RING-NUM,RING-TYPE,SPORE-PRINT-COLOR,POP,HABIT
0,EDIBLE,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,WHITE,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS
1,EDIBLE,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,WHITE,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS
2,EDIBLE,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,PINK,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS
3,EDIBLE,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,PINK,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS
4,EDIBLE,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,BROWN,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS


In [5]:
# use first 5000 data points for training and the rest for test
train_data_m = train_data_m.sample(frac=1,random_state=0).reset_index(drop=True) # random shufle
train_data_m.head()

,CLASS,CAP-SHAPE,CAP-SURF,CAP-COLOR,BRUISES,ODOR,GILL-ATTACH,GILL-SPACE,GILL-SIZE,GILL-COLOR,...,STALK-SURF-BELOW,STALK-COLOR-ABOVE,STALK-COLOR-BELOW,VEIL-TYPE,VEIL-COLOR,RING-NUM,RING-TYPE,SPORE-PRINT-COLOR,POP,HABIT
0,POISONOUS,CONVEX,SCALY,BROWN,NO,SPICY,FREE,CLOSE,NARROW,BUFF,...,SMOOTH,WHITE,PINK,PARTIAL,WHITE,ONE,EVANESCENT,WHITE,SEVERAL,WOODS
1,EDIBLE,CONVEX,FIBROUS,RED,BRUISES,NONE,FREE,CLOSE,BROAD,PURPLE,...,SMOOTH,PINK,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS
2,POISONOUS,FLAT,FIBROUS,GRAY,NO,FOUL,FREE,CLOSE,BROAD,CHOCOLATE,...,SILKY,BUFF,PINK,PARTIAL,WHITE,ONE,LARGE,CHOCOLATE,SOLITARY,GRASSES
3,EDIBLE,FLAT,FIBROUS,BROWN,NO,NONE,FREE,CROWDED,BROAD,BROWN,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,EVANESCENT,BROWN,ABUNDANT,GRASSES
4,EDIBLE,CONVEX,FIBROUS,BROWN,BRUISES,NONE,FREE,CLOSE,BROAD,PURPLE,...,SMOOTH,GRAY,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS


In [6]:
train = train_data_m.iloc[:5000,:]
test = train_data_m.iloc[5000:,:]

## Decision Trees

Make classification with **Decision Trees** using **ID3** (Iterative Dichotomiser 3)

Recall: 

* the decision is built from the dataset and each node is used either to make a decision (internal node) or to represent an outcome (leaves);

* ID3 is a top-down (i.e. the tree is constructed starting from the root) greedy (i.e. we consider only the current step in selecting best features) algorithm to build decision trees;

* at each step features are divided into two or more groups by computing the **information gain**: the feature with the highest information gain is the best one.

  Entropy: $H(S) = \sum_{i=1}^D -p_i \log p_i$, $p_i$ proportion of each category

  Infromation Gain: $IG(S, j)=H(s) - \sum_j \frac{|S_j|}{|S|}H(S_j)$

* a node is selected as leaf if all data in the node belong to the same class;

* repeat until the tree has all leaf nodes (or features are over).




Functions for building the decision tree:

In [7]:
# compute H(S)
def calc_total_entropy(train_data, label, class_list):
    """
    Parameters
    ----------
    train_data : matrix n_data x n_features
        Matrix containing the training dataset
    label : int
        Feature used as label
    class_list : list of str
        Possible values of the label
    """
    total_row = train_data.shape[0]  # the total size of the dataset  
    total_entr = 0
    for c in class_list:  # for each possible class in the label
        total_class_count = train_data[train_data[label] == c].shape[0]  # number of points belonging to the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row)  # entropy of the class
        total_entr += total_class_entr  # adding the class entropy to the total entropy of the dataset
    
    return total_entr

In [8]:
# compute H(S_j)
def calc_entropy(feature_value_data, label, class_list):
    """
    Parameters
    ----------
    feature_value_data : matrix n_data_selected x n_features
        Matrix containing the training points having a certain value of feature j
    label : int
        Feature used as label
    class_list : list of str
        Possible values of the label
    """
    class_count = feature_value_data.shape[0] # n points considered
    entropy = 0
    
    for c in class_list:  # for each possible class in the label
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0]  # row count of class c 
        entropy_class = 0
        if label_class_count != 0:  # avoid numerical errors
            probability_class = label_class_count/class_count  # probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  # entropy
        entropy += entropy_class
    return entropy

In [9]:
# compute information gain in terms of entropy IG(S, j)
def calc_info_gain(feature_name, train_data, label, class_list):
    """
    Parameters
    ----------
    feature_name : str
        Feature considered for computing information gain (j)
    train_data : matrix n_data x n_features
        Matrix containing the training dataset
    label : int
        Feature used as label
    class_list : list of str
        Possible values of the label
    """
    feature_value_list = train_data[feature_name].unique() # unique values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0
    ##### Check for missing values
    a = feature_value_list[feature_value_list=='?']
    t = a.shape[0] # number of points with missing value
    if (t > 0):  # at least a point is missing the entry for this feature
        cmax = 0  # number of points in the most represented class
        n = -1
        for feature_value in feature_value_list:  # possible values for current feature
            n = n + 1
            if (feature_value != '?'):        
                c = train_data[train_data[feature_name] == feature_value].shape[0]
                if (c > cmax):
                    cmax = c
                    fmax = feature_value # value of the feature with the most points
        # replace missing values with the most represented feature value
        train_data[feature_name][:]=[fmax if x=='?' else x for x in train_data[feature_name]]
    #####  now all the data have a value for the feature under analysis
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value]  # filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]  # number of points having feature value in feature j 
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) # calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting

In [11]:
# find feature with maximum information gain
def find_most_informative_feature(train_data, label, class_list):
    """
    Parameters
    ----------
    train_data : matrix n_data x n_features
        Matrix containing the training dataset
    label : int
        Feature used as label
    class_list : list of str
        Possible values of the label
    """
    # N.B. label is not a feature, so drop it!
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset                           
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  # for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: # selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

In [12]:
# split the tree and check finishing condition 
def generate_sub_tree(feature_name, train_data, label, class_list):
    """
    Parameters
    ----------
    feature_name : str
        Feature considered at current node
    train_data : matrix n_data x n_features
        Matrix containing the training dataset
    label : int
        Feature used as label
    class_list : list of str
        Possible values of the label
    """
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False)  # dictionary of the count of unqiue feature value
    tree = {}  # sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value]  # dataset with only feature_name = feature_value
        assigned_to_node = False  # flag for tracking feature_value is pure class or not
        for c in class_list:  # for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0]  # count of class c

            if class_count == count:  # count of (feature_value = count) of class (pure class, any value of the feature represents only one class)
                tree[feature_value] = c  # adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value]  # removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node:  # not pure class
            tree[feature_value] = "?" # as feature_value is not a pure class, it should be expanded further, 
                                      # so the branch is marking with ?
            
    return tree, train_data # in what follows, we have to use this updated dataset

In [13]:
# generate the tree
def make_tree(root, prev_feature_value, train_data, label, class_list):
    """
    Parameters
    ----------
    root : dict 
        Tree written as dictionary of subtrees (initially {})
    prev_feature_value : str
        Previous value of the pointed node (initially None)
    train_data : matrix n_data x n_features
        Matrix containing the training dataset
    label : int
        Feature used as label
    class_list : list of str
        Possible values of the label
    """
    if train_data.shape[0] != 0:  # if dataset becomes empty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list)  # most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list)  # getting tree node and updated dataset
        next_root = None
        if prev_feature_value != None:  # add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree  # expand the tree
            next_root = root[prev_feature_value][max_info_feature]
        else:  # add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()):  # iterating the tree node
            if branch == "?":  # if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node]  # using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list)  # recursive call with updated dataset

In [14]:
# id3 call
def id3(train_data_m, label):
    """
    Parameters
    ----------
    train_data_m : matrix n_data x n_features
        Matrix containing the training dataset
    label : int
        Feature used as label
    """
    train_data = train_data_m.copy()  # getting a copy of the dataset
    tree = {}  # tree which will be updated
    class_list = train_data[label].unique()  # getting unqiue classes of the label
    make_tree(tree, None, train_data, label, class_list)  # start calling recursion
    return tree

Functions for testing the decision tree algorithm:

In [15]:
# prediction from a given instance
def predict(tree, instance):
  # TODO: missing value o label mancante nel training set 
    if not isinstance(tree, dict):  # if it is leaf node
        return tree  # return the value
    else:
        root_node = next(iter(tree))  # getting first key/feature name of the dictionary
        feature_value = instance[root_node]  # value of the feature
        if feature_value in tree[root_node]:  # checking the feature value in current tree node
          if (feature_value != "?"):  # & (feature_value in tree.keys())):
              return predict(tree[root_node][feature_value], instance) # go to next feature
        else:
            return None

In [16]:
# accuracy evaluation
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index in range(len(test_data_m.index)):  # for each row in the dataset
        result = predict(tree, test_data_m.iloc[index])  # predict the row
        if result == test_data_m[label].iloc[index]:  # predicted value and expected value is same or not
            correct_preditct += 1  # increase correct count
        else:
            wrong_preditct += 1  # increase incorrect count
    accuracy = correct_preditct / (correct_preditct + wrong_preditct)  # calculating accuracy
    return accuracy

## Naive Bayes

Make classification using the **Naive Bayes** algorithm.

Recall: 

* Bayes Theorem: $p(class|data) \propto p(data|class)\cdot p(class)$

* prior ($p(class)$) is just the ratio of the number of datapoints belonging to the class;

* to make predictions (i.e. compute the posterior $p(class|data)$) we consider the likelihood of each class ($p(data|class)$) computed as a proportion;

* we work in the log-space so that predictions will be the class maximizing the sum of prior and likelihood.


Function for training the Naive Bayes algorithm:

In [ ]:
def train_naive_bayes(train_data, label):
    """
    Parameters
    ----------
    train_data : matrix n_data x n_features
        Matrix containing the training dataset
    label : int
        Feature used as label
    """
    bayes_pi = {}
    bayes_tab = {}
    ntot = len(train_data.index)
    for cl in train_data[label].unique():  # for each possible value of the label
        fl = train_data[label]  # select training points in current class
        ncl = fl[fl==cl].shape[0]  # count number of points in current class 
        pcl = ncl/ntot # proportion of points in current class (prior)
        bayes_pi[cl] = pcl
    for col in train_data.columns: # for each feature
        if (col != label):  
            dd = pd.crosstab(train_data[label], train_data[col]) # frequency table
            a = np.sum(dd)  # total number of points belonging to a class
            b = np.sum(a[a.keys()!="?"]) 
            bayes_tab[col] = dd/b # likelihhod of each class
    
    return bayes_pi, bayes_tab

Function for testing Naive Bayes:

In [ ]:
# prediction and accuracy evaluation
def predict_naive_bayes(test_data, bayes_pi, bayes_tab, label):
    ntot = len(test_data.index)
    ncorrect = 0
    for j in range(ntot):            
        prob = bayes_pi.copy()
        for col in test_data.columns:
            if (col != label):
                if ((test_data[col].iloc[j]!="?")&(test_data[col].iloc[j] in bayes_tab[col].keys())):
                    for cl in bayes_pi.keys():
                        prob[cl] = prob[cl]*bayes_tab[col][test_data[col].iloc[j]][cl]
        if (test_data[label].iloc[j] == max(prob, key=prob.get)):
            ncorrect = ncorrect + 1
    return (ncorrect/ntot) # accuracy

## Results on Mushroom Dataset

### Decision Tree

In [17]:
tree = id3(train, "CLASS")
print(tree)

{'ODOR': {'SPICY': 'POISONOUS', 'NONE': {'SPORE-PRINT-COLOR': {'BROWN': 'EDIBLE', 'BLACK': 'EDIBLE', 'WHITE': {'HABIT': {'GRASSES': 'EDIBLE', 'WASTE': 'EDIBLE', 'LEAVES': {'CAP-COLOR': {'BROWN': 'EDIBLE', 'CINNAMON': 'EDIBLE', 'YELLOW': 'POISONOUS', 'WHITE': 'POISONOUS'}}, 'WOODS': {'GILL-SIZE': {'BROAD': 'EDIBLE', 'NARROW': 'POISONOUS'}}, 'PATHS': 'EDIBLE'}}, 'ORANGE': 'EDIBLE', 'BUFF': 'EDIBLE', 'YELLOW': 'EDIBLE', 'GREEN': 'POISONOUS', 'CHOCOLATE': 'EDIBLE'}}, 'FOUL': 'POISONOUS', 'PUNGENT': 'POISONOUS', 'FISHY': 'POISONOUS', 'ANISE': 'EDIBLE', 'CREOSOTE': 'POISONOUS', 'ALMOND': 'EDIBLE', 'MUSTY': 'POISONOUS'}}


In [18]:
accuracy = evaluate(tree, test, "CLASS")
print(accuracy)

1.0


### Naive Bayes

In [19]:
pi, tab = train_naive_bayes(train, "CLASS")

NameError: name 'train_naive_bayes' is not defined

In [ ]:
accuracy = predict_naive_bayes(test, pi, tab,"CLASS")
print(accuracy)